# Multi-Site Aethalometer Analysis (Modular Version)

This notebook uses the modularized scripts for multi-site aethalometer and filter data analysis.

**Sites:**
- Beijing, China (CHTS)
- Delhi, India (INDH)
- JPL/Pasadena, USA (USPA)
- Addis Ababa, Ethiopia (ETAD)

---

## 1. Setup and Imports

In [ ]:
# Add scripts folder to path
import sys
sys.path.insert(0, './scripts')

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import from our modular scripts
from config import (
    SITES, PROCESSED_SITES_DIR, FILTER_DATA_PATH,
    MAC_VALUE, FLOW_FIX_PERIODS, MIN_EC_THRESHOLD,
    SMOOTH_RAW_THRESHOLDS, CROSS_COMPARISONS
)

from outliers import (
    EXCLUDED_SAMPLES, MANUAL_OUTLIERS,
    apply_exclusion_flags, apply_threshold_flags,
    get_clean_data, print_exclusion_summary,
    identify_outlier_dates
)

from data_matching import (
    load_aethalometer_data, load_filter_data,
    match_aeth_filter_data, match_all_parameters,
    match_with_smooth_raw_info, add_flow_period_column,
    get_site_code, get_site_color, print_data_summary
)

from plotting import (
    calculate_regression_stats,
    plot_crossplot, plot_before_after_comparison,
    create_tiled_threshold_plots, plot_smooth_raw_distribution,
    plot_bc_timeseries, plot_multiwavelength_bc,
    print_comparison_table
)

# Configure matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Modules loaded successfully!")
print(f"\nSites configured: {list(SITES.keys())}")
print(f"MAC value: {MAC_VALUE} m^2/g")

## 2. Load Data

In [ ]:
# Load all aethalometer datasets
aethalometer_data = load_aethalometer_data()

In [ ]:
# Load filter dataset
filter_data = load_filter_data()

In [ ]:
# Print data summary
print_data_summary(aethalometer_data, filter_data)

## 3. BC Time Series

In [ ]:
# Combined BC time series - all sites
fig, ax = plt.subplots(figsize=(14, 7))

for site_name, df in aethalometer_data.items():
    config = SITES[site_name]
    plot_bc_timeseries(ax, site_name, df, config, wavelength='IR')

ax.set_title('IR Black Carbon Time Series - All Sites', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Individual site time series
for site_name, df in aethalometer_data.items():
    config = SITES[site_name]
    
    fig, ax = plt.subplots(figsize=(12, 5))
    plot_bc_timeseries(ax, site_name, df, config, wavelength='IR')
    
    ax.set_title(f'IR Black Carbon - {site_name}', fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 4. Aethalometer vs FTIR EC Cross-Plots

Compare aethalometer BC with filter-based FTIR EC measurements.

In [ ]:
# Match data and create cross-plots for each site
all_matched_data = {}

for site_name in SITES:
    if site_name not in aethalometer_data:
        continue
    
    config = SITES[site_name]
    df_aeth = aethalometer_data[site_name]
    
    # Match aethalometer and filter data
    matched = match_aeth_filter_data(
        site_name, df_aeth, filter_data, config['code']
    )
    
    if matched is not None and len(matched) >= 3:
        all_matched_data[site_name] = matched
        print(f"{site_name}: {len(matched)} matched pairs")
    else:
        print(f"{site_name}: Insufficient matched data")

In [ ]:
# Combined scatter plot - all sites
fig, ax = plt.subplots(figsize=(10, 10))

for site_name, matched_df in all_matched_data.items():
    config = SITES[site_name]
    ax.scatter(
        matched_df['aeth_bc'], matched_df['filter_ec'],
        color=config['color'], alpha=0.6, s=80,
        edgecolors='black', linewidth=1,
        label=f"{site_name} (n={len(matched_df)})"
    )

# Set axes to start from 0
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)

# Add 1:1 line
max_val = max(ax.get_xlim()[1], ax.get_ylim()[1])
ax.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, linewidth=1.5, label='1:1 line')

ax.set_xlabel('Aethalometer IR BC (ng/m³)', fontsize=12)
ax.set_ylabel('FTIR EC (ng/m³)', fontsize=12)
ax.set_title('Aethalometer BC vs FTIR EC - All Sites', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Individual cross-plots with regression stats
site_stats = {}

for site_name, matched_df in all_matched_data.items():
    config = SITES[site_name]
    
    fig, ax = plt.subplots(figsize=(9, 9))
    
    stats = plot_crossplot(
        ax,
        matched_df['aeth_bc'].values,
        matched_df['filter_ec'].values,
        'Aethalometer IR BC (ng/m³)',
        'FTIR EC (ng/m³)',
        color=config['color'],
        equal_axes=True
    )
    
    ax.set_title(f'{site_name}: Aethalometer BC vs FTIR EC', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    if stats:
        site_stats[site_name] = stats
        print(f"{site_name}: R² = {stats['r_squared']:.3f}, Slope = {stats['slope']:.3f}\n")

## 5. Outlier Detection and Removal

Apply manual outlier exclusions and see the impact on regression statistics.

In [ ]:
# Show current outlier configuration
print("Current EXCLUDED_SAMPLES registry:")
print("="*60)
for site, exclusions in EXCLUDED_SAMPLES.items():
    print(f"\n{site}: {len(exclusions)} exclusion(s)")
    for excl in exclusions:
        print(f"  - {excl['date']}: {excl['reason'][:50]}...")

In [ ]:
# Find actual outlier dates from your data
# Run this to identify dates to add to EXCLUDED_SAMPLES in outliers.py

print("Finding outlier dates based on MANUAL_OUTLIERS thresholds...")
print("="*60)

for site_name, matched_df in all_matched_data.items():
    criteria = MANUAL_OUTLIERS.get(site_name, {}).get('remove_criteria', [])
    
    if not criteria:
        print(f"\n{site_name}: No thresholds defined")
        continue
    
    print(f"\n{site_name}:")
    for crit in criteria:
        identify_outlier_dates(site_name, matched_df, crit)

In [ ]:
# Apply threshold-based outlier flagging and create before/after plots (separate)

for site_name, matched_df in all_matched_data.items():
    config = SITES[site_name]
    
    # Apply threshold-based outlier flags
    matched_with_flags = apply_threshold_flags(matched_df.copy(), site_name)
    
    # Print summary
    print_exclusion_summary(matched_with_flags, site_name)
    
    # Check if there are any outliers
    if not matched_with_flags['is_outlier'].any():
        print(f"  No outliers flagged for {site_name}")
        
        # Still show the single plot
        fig, ax = plt.subplots(figsize=(9, 9))
        stats = plot_crossplot(
            ax,
            matched_with_flags['aeth_bc'].values,
            matched_with_flags['filter_ec'].values,
            'Aethalometer IR BC (ng/m³)',
            'FTIR EC (ng/m³)',
            color=config['color'],
            equal_axes=True
        )
        ax.set_title(f'{site_name}: Aethalometer BC vs FTIR EC (No Outliers)', 
                     fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        print()
        continue
    
    # Calculate axis limits (same for both plots)
    all_vals = np.concatenate([
        matched_with_flags['aeth_bc'].dropna().values,
        matched_with_flags['filter_ec'].dropna().values
    ])
    max_val = all_vals.max() * 1.1
    
    x_data = matched_with_flags['aeth_bc'].values
    y_data = matched_with_flags['filter_ec'].values
    outlier_mask = matched_with_flags['is_outlier'].values
    
    # ---- BEFORE: All Data ----
    fig, ax = plt.subplots(figsize=(9, 9))
    
    # Plot all points
    ax.scatter(x_data, y_data, color=config['color'], alpha=0.6, s=80,
               edgecolors='black', linewidth=1, label=f'All data (n={len(x_data)})')
    
    # Highlight outliers with red circles
    ax.scatter(x_data[outlier_mask], y_data[outlier_mask],
               facecolors='none', edgecolors='red', s=200, linewidths=3,
               label=f'Outliers (n={outlier_mask.sum()})')
    
    # Regression on ALL data
    stats_all = calculate_regression_stats(x_data, y_data)
    if stats_all:
        x_line = np.array([0, max_val])
        y_line = stats_all['slope'] * x_line + stats_all['intercept']
        ax.plot(x_line, y_line, 'r-', linewidth=2, alpha=0.8, label='Best fit (all)')
        
        sign = '+' if stats_all['intercept'] >= 0 else '-'
        eq = f"y = {stats_all['slope']:.3f}x {sign} {abs(stats_all['intercept']):.2f}"
        stats_text = f"ALL DATA\nn = {stats_all['n']}\nR² = {stats_all['r_squared']:.3f}\n{eq}"
        ax.text(0.05, 0.95, stats_text, transform=ax.transAxes, fontsize=10,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
    
    ax.set_xlim(0, max_val)
    ax.set_ylim(0, max_val)
    ax.set_aspect('equal', adjustable='box')
    ax.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, linewidth=1.5, label='1:1 line')
    ax.set_xlabel('Aethalometer IR BC (ng/m³)', fontsize=11)
    ax.set_ylabel('FTIR EC (ng/m³)', fontsize=11)
    ax.set_title(f'{site_name}: BEFORE - All Data (Outliers Highlighted)', 
                 fontsize=14, fontweight='bold')
    ax.legend(loc='lower right', fontsize=9)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # ---- AFTER: Outliers Removed ----
    fig, ax = plt.subplots(figsize=(9, 9))
    
    # Plot clean data
    x_clean = x_data[~outlier_mask]
    y_clean = y_data[~outlier_mask]
    ax.scatter(x_clean, y_clean, color=config['color'], alpha=0.6, s=80,
               edgecolors='black', linewidth=1, label=f'Retained (n={len(x_clean)})')
    
    # Show removed points as red X
    ax.scatter(x_data[outlier_mask], y_data[outlier_mask],
               color='red', alpha=0.9, s=200, marker='X', linewidths=3,
               label=f'Removed (n={outlier_mask.sum()})')
    
    # Regression on CLEAN data
    stats_clean = calculate_regression_stats(x_clean, y_clean)
    if stats_clean:
        x_line = np.array([0, max_val])
        y_line = stats_clean['slope'] * x_line + stats_clean['intercept']
        ax.plot(x_line, y_line, 'g-', linewidth=2, alpha=0.8, label='Best fit (clean)')
        
        sign = '+' if stats_clean['intercept'] >= 0 else '-'
        eq = f"y = {stats_clean['slope']:.3f}x {sign} {abs(stats_clean['intercept']):.2f}"
        stats_text = f"CLEANED DATA\nn = {stats_clean['n']}\nR² = {stats_clean['r_squared']:.3f}\n{eq}"
        ax.text(0.05, 0.95, stats_text, transform=ax.transAxes, fontsize=10,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.9))
    
    ax.set_xlim(0, max_val)
    ax.set_ylim(0, max_val)
    ax.set_aspect('equal', adjustable='box')
    ax.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, linewidth=1.5, label='1:1 line')
    ax.set_xlabel('Aethalometer IR BC (ng/m³)', fontsize=11)
    ax.set_ylabel('FTIR EC (ng/m³)', fontsize=11)
    ax.set_title(f'{site_name}: AFTER - Outliers Removed', 
                 fontsize=14, fontweight='bold')
    ax.legend(loc='lower right', fontsize=9)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Print impact
    if stats_all and stats_clean:
        print(f"\n  Impact of outlier removal:")
        print(f"    R²: {stats_all['r_squared']:.3f} → {stats_clean['r_squared']:.3f} "
              f"(Δ = {stats_clean['r_squared'] - stats_all['r_squared']:+.3f})")
        print(f"    Slope: {stats_all['slope']:.3f} → {stats_clean['slope']:.3f}")
    
    print()

## 6. Smooth vs Raw BC Analysis

Analyze the impact of smooth/raw BC differences on data quality.

In [ ]:
# Match data with smooth/raw info
smooth_raw_data = {}

for site_name in SITES:
    if site_name not in aethalometer_data:
        continue
    
    config = SITES[site_name]
    df_aeth = aethalometer_data[site_name]
    
    matched = match_with_smooth_raw_info(
        site_name, df_aeth, filter_data, config['code']
    )
    
    if matched is not None and len(matched) >= 3:
        n_with_smooth = matched['smooth_raw_abs_pct'].notna().sum()
        smooth_raw_data[site_name] = matched
        print(f"{site_name}: {len(matched)} matched, {n_with_smooth} with smooth data")
    else:
        print(f"{site_name}: Insufficient data")

In [ ]:
# Distribution plots for each site (separate histogram and cumulative)
for site_name, matched_df in smooth_raw_data.items():
    config = SITES[site_name]
    
    valid_diffs = matched_df['smooth_raw_abs_pct'].dropna()
    
    if len(valid_diffs) < 3:
        print(f"{site_name}: Insufficient smooth data")
        continue
    
    print(f"\n{'='*60}")
    print(f"{site_name}: Smooth/Raw Difference Distribution")
    print(f"{'='*60}")
    print(f"  n = {len(valid_diffs)}")
    print(f"  Mean: {valid_diffs.mean():.2f}%")
    print(f"  Median: {valid_diffs.median():.2f}%")
    print(f"  Range: {valid_diffs.min():.2f}% - {valid_diffs.max():.2f}%")
    
    threshold_colors = ['green', 'blue', 'orange', 'red']
    
    # ---- Histogram ----
    fig, ax = plt.subplots(figsize=(10, 6))
    
    bins = np.arange(0, valid_diffs.max() + 1, 0.5)
    ax.hist(valid_diffs, bins=bins, color=config['color'], alpha=0.7,
            edgecolor='black', linewidth=0.5)
    
    for threshold, color in zip(SMOOTH_RAW_THRESHOLDS, threshold_colors):
        n_kept = (valid_diffs <= threshold).sum()
        ax.axvline(x=threshold, color=color, linestyle='--', linewidth=2,
                   label=f'{threshold}%: {n_kept} kept')
    
    ax.set_xlabel('Absolute % Difference (Smooth - Raw)', fontsize=11)
    ax.set_ylabel('Count', fontsize=11)
    ax.set_title(f'{site_name}: Distribution of Smooth/Raw Differences',
                 fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # ---- Cumulative Distribution ----
    fig, ax = plt.subplots(figsize=(10, 6))
    
    sorted_diffs = np.sort(valid_diffs)
    cumulative = np.arange(1, len(sorted_diffs) + 1) / len(sorted_diffs) * 100
    
    ax.plot(sorted_diffs, cumulative, color=config['color'], linewidth=2)
    ax.fill_between(sorted_diffs, cumulative, alpha=0.3, color=config['color'])
    
    for threshold, color in zip(SMOOTH_RAW_THRESHOLDS, threshold_colors):
        pct_below = (valid_diffs <= threshold).sum() / len(valid_diffs) * 100
        ax.axvline(x=threshold, color=color, linestyle='--', linewidth=2)
        ax.annotate(f'{threshold}%: {pct_below:.1f}% kept',
                    xy=(threshold, pct_below), xytext=(threshold + 0.5, pct_below - 5),
                    fontsize=9, color=color)
    
    ax.set_xlabel('Absolute % Difference (Smooth - Raw)', fontsize=11)
    ax.set_ylabel('Cumulative % of Data Points', fontsize=11)
    ax.set_title(f'{site_name}: Cumulative Distribution',
                 fontsize=14, fontweight='bold')
    ax.set_ylim(0, 105)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Print threshold summary
    print(f"\n  Points kept at each threshold:")
    for threshold in SMOOTH_RAW_THRESHOLDS:
        n_kept = (valid_diffs <= threshold).sum()
        pct_kept = n_kept / len(valid_diffs) * 100
        print(f"    <={threshold}%: {n_kept}/{len(valid_diffs)} ({pct_kept:.1f}%)")

In [ ]:
# Individual threshold plots (separate plots for each threshold)
threshold_results = {}

for site_name, matched_df in smooth_raw_data.items():
    config = SITES[site_name]
    
    if matched_df['smooth_raw_abs_pct'].notna().sum() < 3:
        print(f"{site_name}: Insufficient smooth data")
        continue
    
    print(f"\n{'='*60}")
    print(f"{site_name}: Smooth/Raw Threshold Analysis")
    print(f"{'='*60}")
    
    site_results = {}
    
    # Calculate axis limits (same for all plots for this site)
    all_vals = np.concatenate([
        matched_df['aeth_bc'].dropna().values,
        matched_df['filter_ec'].dropna().values
    ])
    max_val = all_vals.max() * 1.1 if len(all_vals) > 0 else 100
    
    for threshold in SMOOTH_RAW_THRESHOLDS:
        # Separate by threshold
        below_threshold = matched_df[matched_df['smooth_raw_abs_pct'] <= threshold].copy()
        above_threshold = matched_df[matched_df['smooth_raw_abs_pct'] > threshold].copy()
        
        n_kept = len(below_threshold)
        n_removed = len(above_threshold)
        
        if n_kept < 3:
            print(f"  Threshold <={threshold}%: Insufficient data (n={n_kept})")
            continue
        
        # Create individual figure for this threshold
        fig, ax = plt.subplots(figsize=(9, 9))
        
        # Plot kept points
        ax.scatter(below_threshold['aeth_bc'], below_threshold['filter_ec'],
                   color=config['color'], alpha=0.6, s=80,
                   edgecolors='black', linewidth=1,
                   label=f'<={threshold}% diff (n={n_kept})')
        
        # Plot removed points as gray X's
        if len(above_threshold) > 0:
            ax.scatter(above_threshold['aeth_bc'], above_threshold['filter_ec'],
                       color='gray', alpha=0.5, s=120, marker='X',
                       linewidths=2, label=f'>{threshold}% diff (n={n_removed})')
        
        # Calculate regression on kept points
        stats = calculate_regression_stats(
            below_threshold['aeth_bc'].values,
            below_threshold['filter_ec'].values
        )
        
        if stats:
            # Plot regression line
            x_line = np.array([0, max_val])
            y_line = stats['slope'] * x_line + stats['intercept']
            ax.plot(x_line, y_line, 'g-', linewidth=2, alpha=0.8, label='Best fit')
            
            # Stats text
            sign = '+' if stats['intercept'] >= 0 else '-'
            eq = f"y = {stats['slope']:.3f}x {sign} {abs(stats['intercept']):.2f}"
            stats_text = f"n = {stats['n']}\nR² = {stats['r_squared']:.3f}\n{eq}"
            
            ax.text(0.05, 0.95, stats_text, transform=ax.transAxes, fontsize=10,
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.9))
            
            site_results[threshold] = {
                'n_kept': n_kept,
                'n_removed': n_removed,
                **stats
            }
        
        # Set axes
        ax.set_xlim(0, max_val)
        ax.set_ylim(0, max_val)
        ax.set_aspect('equal', adjustable='box')
        ax.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, linewidth=1.5, label='1:1 line')
        
        ax.set_xlabel('Aethalometer BC (ng/m³)', fontsize=11)
        ax.set_ylabel('FTIR EC (ng/m³)', fontsize=11)
        ax.set_title(f'{site_name}: Threshold <={threshold}% Smooth-Raw Difference',
                     fontsize=14, fontweight='bold')
        ax.legend(loc='lower right', fontsize=9)
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        if stats:
            print(f"  Threshold <={threshold}%: R² = {stats['r_squared']:.3f}, n = {stats['n']}")
    
    threshold_results[site_name] = site_results

In [ ]:
# Summary table
print_comparison_table(threshold_results, metric_name='Smooth/Raw Threshold Impact')

## 7. Cross-Comparisons: HIPS, FTIR EC, Iron, Aethalometer

Compare different measurement methods across all sites.

In [ ]:
# Match all parameters for each site
all_params_data = {}

for site_name in SITES:
    if site_name not in aethalometer_data:
        continue
    
    config = SITES[site_name]
    df_aeth = aethalometer_data[site_name]
    
    matched = match_all_parameters(
        site_name, config['code'], df_aeth, filter_data
    )
    
    if matched is not None and len(matched) >= 3:
        all_params_data[site_name] = matched
        
        # Show available parameters
        available = [col for col in ['ir_bcc', 'hips_fabs', 'ftir_ec', 'iron'] 
                     if col in matched.columns and matched[col].notna().any()]
        print(f"{site_name}: {len(matched)} days, params: {', '.join(available)}")
    else:
        print(f"{site_name}: Insufficient data")

In [ ]:
# Create cross-comparison plots with outlier exclusions
comparison_results = {}

for site_name, matched_df in all_params_data.items():
    config = SITES[site_name]
    site_results = {}
    
    print(f"\n{'='*60}")
    print(f"{site_name}")
    print(f"{'='*60}")
    
    # Apply date-based exclusions from EXCLUDED_SAMPLES
    matched_df = apply_exclusion_flags(matched_df, site_name)
    
    # Print exclusion summary
    n_excluded = matched_df['is_excluded'].sum() if 'is_excluded' in matched_df.columns else 0
    if n_excluded > 0:
        print(f"  Excluding {n_excluded} outlier(s)")
    
    for comp in CROSS_COMPARISONS:
        x_col = comp['x_col']
        y_col = comp['y_col']
        
        # Check if columns exist
        if x_col not in matched_df.columns or y_col not in matched_df.columns:
            continue
        
        x_data = matched_df[x_col].values
        y_data = matched_df[y_col].values
        
        # Get outlier mask
        outlier_mask = matched_df['is_excluded'].values if 'is_excluded' in matched_df.columns else None
        
        # Check for valid data
        valid_count = ((~np.isnan(x_data)) & (~np.isnan(y_data))).sum()
        if valid_count < 3:
            continue
        
        fig, ax = plt.subplots(figsize=(9, 9))
        
        stats = plot_crossplot(
            ax, x_data, y_data,
            comp['x_label'], comp['y_label'],
            color=config['color'],
            equal_axes=comp['equal_axes'],
            show_mac=comp['show_mac'],
            outlier_mask=outlier_mask  # Pass the outlier mask
        )
        
        ax.set_title(f"{site_name}: {comp['name']}", fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        if stats:
            site_results[comp['name']] = stats
            print(f"  {comp['name']}: R² = {stats['r_squared']:.3f}, n = {stats['n']}")
    
    comparison_results[site_name] = site_results

## 8. Summary Statistics

In [ ]:
# Create summary table of all cross-comparisons
print("\n" + "="*100)
print("SUMMARY: CROSS-COMPARISON RESULTS")
print("="*100)
print(f"\nNote: HIPS Fabs divided by MAC = {MAC_VALUE} m²/g")

for comp in CROSS_COMPARISONS:
    comp_name = comp['name']
    print(f"\n{comp_name}:")
    print(f"{'Site':<15s} {'n':>8s} {'R²':>10s} {'Slope':>10s}")
    print("-" * 45)
    
    for site_name in SITES:
        if site_name in comparison_results and comp_name in comparison_results[site_name]:
            stats = comparison_results[site_name][comp_name]
            print(f"{site_name:<15s} {stats['n']:>8d} {stats['r_squared']:>10.3f} {stats['slope']:>10.3f}")
        else:
            print(f"{site_name:<15s} {'--':>8s} {'--':>10s} {'--':>10s}")

In [ ]:
# BC statistics by site
print("\n" + "="*80)
print("BC CONCENTRATION STATISTICS BY SITE")
print("="*80)

for site_name, df in aethalometer_data.items():
    if 'IR BCc' in df.columns:
        bc_data = df['IR BCc'].dropna()
        
        print(f"\n{site_name}:")
        print(f"  n = {len(bc_data)}")
        print(f"  Mean: {bc_data.mean():.1f} ng/m³")
        print(f"  Median: {bc_data.median():.1f} ng/m³")
        print(f"  Std: {bc_data.std():.1f} ng/m³")
        print(f"  Range: {bc_data.min():.1f} - {bc_data.max():.1f} ng/m³")

---

## Notes

### Updating Outliers

To add new outliers:

1. Run `identify_outlier_dates()` to find exact dates
2. Edit `scripts/outliers.py` and add entries to `EXCLUDED_SAMPLES`
3. Restart kernel and re-run

### Configuration

Key settings are in `scripts/config.py`:
- `SITES` - Site definitions (codes, colors, files)
- `MAC_VALUE` - Mass Absorption Cross-section (default: 10)
- `MIN_EC_THRESHOLD` - Minimum EC to include (default: 0.5 µg/m³)
- `FLOW_FIX_PERIODS` - Before/after flow fix dates

### Module Structure

```
scripts/
├── config.py        - Central configuration
├── outliers.py      - Outlier registry and functions
├── data_matching.py - Data loading and matching
└── plotting.py      - Reusable plotting functions
```